In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [2]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# Define the path to your PDF files
pdf_dir = "./docs"  # Cambia esto al directorio donde tienes tus PDFs

# Load PDF documents
pdf_files = [os.path.join(pdf_dir, file) for file in os.listdir(pdf_dir) if file.endswith(".pdf")]
docs = [PyPDFLoader(pdf).load() for pdf in pdf_files]

# Convert list of lists to a single list of documents
docs_list = [item for sublist in docs for item in sublist]

In [3]:
# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=100)
doc_splits = text_splitter.split_documents(docs_list) 

In [4]:
len(doc_splits)

45

: 

In [5]:
from langchain_community.vectorstores import Chroma

# Define el directorio donde se almacenará la base de datos
persist_directory = "./chroma_db"  # Puedes cambiar esto al directorio que prefieras

# Crear la base de datos de vectores local
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(),
    persist_directory=persist_directory  # Especifica el directorio de persistencia
)

# # Cargar la base de datos de vectores desde el directorio local
# vectorstore = Chroma(
#     collection_name="rag-chroma",
#     embedding=OpenAIEmbeddings(),
#     persist_directory=persist_directory  # Especifica el directorio de persistencia
# )

In [6]:
retriever = vectorstore.as_retriever()

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_company_docs",
    "Search and return information about the company documents",
)

In [7]:
question = "politica de atencion al cliente"
documents = retriever.invoke(question)
documents

[Document(metadata={'page': 0, 'source': './docs\\OSPE_Documentacion.pdf'}, page_content='OSPE se compromete a ofrecer un servicio de atención al cliente excepcional, asegurando que\ntodos nuestros afiliados reciban el soporte y la información necesarios de manera eficiente y\nprofesional.'),
 Document(metadata={'page': 0, 'source': './docs\\OSPE_Documentacion.pdf'}, page_content='OSPE se compromete a ofrecer un servicio de atención al cliente excepcional, asegurando que\ntodos nuestros afiliados reciban el soporte y la información necesarios de manera eficiente y\nprofesional.'),
 Document(metadata={'page': 2, 'source': './docs\\OSPE_Documentacion.pdf'}, page_content='OSPE fue fundada como una pequeña aseguradora de salud local y ha crecido hasta convertirse\nen una empresa con cobertura nacional. A lo largo de los años, hemos expandido nuestra red de'),
 Document(metadata={'page': 2, 'source': './docs\\OSPE_Documentacion.pdf'}, page_content='OSPE fue fundada como una pequeña asegurad